In [73]:
from medbert.visualization.visualize import BertViz

In [70]:
patient_num = 10
visualizer = BertViz("..\\models\\mlm_plos_pretrained\\test\\model.pt", 
        "..\\data\\tokenized\\example_data.pt")

In [74]:
visualizer.head_view(patient_num)

<IPython.core.display.Javascript object>

In [72]:
visualizer.model_view(patient_num)

<IPython.core.display.Javascript object>